In [1]:
import os
import pyvtk
import numpy as np
import xarray as xr

In [2]:
# data dir
datadir = 'output/stations/inplane_stations'

# open 
nc = xr.open_mfdataset(datadir + '/axisem3d_synthetics.nc.rank*', 
                       concat_dim='dim_station', combine='nested', 
                       data_vars=['data_wave', 'station_order'])

# read time
time_points = nc.data_vars['time_points'].values

# read wave
data_wave = nc.data_vars['data_wave'].values

# read station order
station_order = nc.data_vars['station_order'].values.astype(str)

# close
nc.close()

In [3]:
# inverse map of station_order
dict_key_to_loc = {}
for loc, key in enumerate(station_order):
    dict_key_to_loc[key] = loc
    
# dimensions
nstation = data_wave.shape[0]
ntime = data_wave.shape[2]

In [4]:
###### vtk grid ######

# z coord on surface
surface_z = 6370e3

# x coord of stations
x_all = np.arange(0, 5000, 50)

# depth of station
depth_all = np.arange(0, 3801, 50)

# grid xyz and connectivity
xyz = np.ndarray((nstation, 3))
connect = []

for ix, x in enumerate(x_all):
    for idep, dep in enumerate(depth_all):
        # coords
        loc = dict_key_to_loc['INPLANE.st_x%d_d%d' % (x, dep)]
        xyz[loc, :] = [x, 0, surface_z - dep]
        # connectivity
        if ix != len(x_all) - 1 and idep != len(depth_all) - 1:
            connect.append([loc, 
                            dict_key_to_loc['INPLANE.st_x%d_d%d' % (x + 50, dep)], 
                            dict_key_to_loc['INPLANE.st_x%d_d%d' % (x + 50, dep + 50)], 
                            dict_key_to_loc['INPLANE.st_x%d_d%d' % (x, dep + 50)]])
connect = np.array(connect)

# points
vtk_points = pyvtk.UnstructuredGrid(list(zip(xyz[:,0], xyz[:,1], xyz[:,2])), quad=connect)


############ vtk ############
os.makedirs('%s/vtk' % datadir, exist_ok=True)

for istep in np.arange(ntime):
    # vtk
    vtk = pyvtk.VtkData(vtk_points, 
                        pyvtk.PointData(pyvtk.Scalars(data_wave[:, 0, istep], name='U_NORM')))
    vtk.tofile('%s/vtk/U_NORM.%d.vtk' % (datadir, istep), 'binary')